<a href="https://colab.research.google.com/github/Ali-Sab/aps360_project/blob/main/APS360_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
print("Welcome!")
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt # for plotting
import torch.optim as optim
print("welcome again")
print("Trying to add stuff and also take it away")
print("Also merging again")

Welcome!
welcome again
A change is made after github is created to test github


djdjdjdjdjdjdjdjdjdj

Hi this is Ammol's commit

In [ ]:
#I i would like to observe what happens now.
I JUST GOT A MERGE CONFLICT AND FIXED IT
# Hello everyone. I would like to point out that this is a test. Do not be confused. :D 